# 総合添削問題

　以下の画像の色を減らし、N色で構成された画像に変換することを考えます。このような場合、クラスタリングによりこの問題を解決することが可能です。

<img src="./img_cluster/img_1_0.png">

　上記の画像を32色にまで減色すると以下のようになります。

<img src="./img_cluster/output32.bmp">

具体的な流れは、以下のようになります。
画像の中で代表的な色を数種類選ぶ。
画像中の全ピクセルを一番色が近い1.で選ばれた色に置き換える。
2.における色の近さの定義は、画像中の色は(R, G, B)=(それぞれ0~255)で表されるため、RGBの値の大きさの3次元のユークリッド距離で算出されます。

　　　　 $(R1,G1,B1)と(R2,G2,B2)の距離=(R1−R2)^2+(G1−G2)^2+(B1−B2)^2$
 
1.での代表的な色の選択は、クラスタリング手法であるk-means法を用います。
はじめに、ランダムにN個の代表色を選択します。
画像上の全ピクセルにおいて、一番近い代表色を選び、全ピクセルをN個のクラスターに割り当てます。
それぞれのクラスターにおける平均色(R,G,Bのそれぞれの平均値をとった値)を算出します。
新しい代表色を用いて、再度2.を繰り返します。
これを繰り返すと、各クラスターの平均値が代表色の値と一致し、代表色の変化が止まります。これを最終的な代表色として割り当てます。

#### 問題

- 上記のアルゴリズムを理解し、画像とNを入力されると、画像をN色で構成される画像に変換するプログラムを構築してください。

In [15]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randint
from PIL import Image
import cv2

#------------#
# パラメータ #
#------------#
# 減色後の色数（任意の個数の色数を指定できます）
Colors = [32]  


# k平均法による減色処理
def run_kmeans(pixels, k):
    cls = [0] * len(pixels)

    # 代表色の初期値をランダムに設定
    center = []
    # randintにより[R, G, B]をそれぞれ0~255の範囲でk個生成
    for i in range(k):
        center.append(np.array([randint(256), randint(256), randint(256)]))
    # numpy.arrayをlist型に変換
    print (map(lambda x: x.tolist(), center))
    
    distortion = 0

    # 最大50回のIterationを実施
    for iter_num in range(50): 
        # 各代表色のRBG値を保存するための空listを作成
        center_new = []
        for i in range(k):
            center_new.append(np.array([0,0,0]))
        
        num_points = [0] * k
#        print(num_points)
        distortion_new = 0

        # E Phase: 各データが属するグループ（代表色）を計算
        for pix, point in enumerate(pixels):
            min_dist = 256*256*3
            point = np.array(point) 
            
            # データ群の中からk個(任意の数)のデータ点を抽出し、 そのk個の点を初期のセントロイドとします。 
            for i in range(k):
                # ピクセルの座標におけるRGB値と代表色のユークリッド距離を算出
                d = np.linalg.norm(point - center[i])
                # dの値が設定した最小値より小さければ最小値にdを代入し、
                # 各ピクセルの代表色の番号をリストclsに保存
                if d < min_dist:
                    min_dist = d
                    cls[pix] = i
                
            # データ点に最も近い中心点の、所属データ件数に１加算
            cls_idx = cls[pix]
            num_points[cls_idx] += 1
            # 重心を計算するため、上記中心点にRGB値を保持しておく
            center_new[cls_idx] += point
            # 中心点との距離の合計を更新（中心点との距離合計の変化量が0.5%未満になったら計算終了する番兵を実装するため、計算しておく）
            distortion_new += min_dist
        
        # M Phase: 新しい代表色を求める
        for i in range(k):
            # グループごとのデータ点から重心を計算する
            center[i] = center_new[i] / num_points[i]

        print (map(lambda x: x.tolist(), center))
        print ("Distortion = %d" % distortion_new)

        # Distortion(J)の変化が0.5%未満になったら終了
        if iter_num > 0 and distortion - distortion_new < distortion * 0.005:
            break
        distortion = distortion_new

    # 画像データの各ピクセルを代表色で置き換え
    for pix, point in enumerate(pixels):
        pixels[pix] = tuple(map(lambda x: int(x), center[cls[pix]]))

    return pixels
        
# 任意のデータ点を抽出
for k in Colors:
    print ("k=%d" % k)
    # 画像ファイルの読み込み
    im = Image.open("./img_cluster/img_1_0.png")
    pixels = list(im.convert('RGB').getdata())
    # k平均法による減色処理
    result = run_kmeans(pixels, k)
    # 画像データの更新とファイル出力
    im.putdata(result) # Update image
    im.save("output%02d.bmp" % k, "BMP")


k=32


/usr/local/var/pyenv/versions/anaconda3-5.2.0/envs/py3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in true_divide


Distortion = 3084750
Distortion = 1662249
Distortion = 1398362
Distortion = 1187444
Distortion = 1116256
Distortion = 1064409
Distortion = 1036880
Distortion = 1027441
Distortion = 1024334


#### ヒント

- k-meansを実装する問題になります、わからなければ復習しましょう。

##  解答例

添削課題の提出は以下のアドレスから提出いただきますようお願いします。<br>

https://goo.gl/forms/fW7CAspZMwHuWuqk2<br><br>
以下のアドレスからアンケートにご協力頂きたく存じます。<br>
ご回答のほど、よろしくお願いいたします。

https://goo.gl/forms/WHjJQYeodIndRvyz2